In [2]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle

In [3]:
from feeds import BackFeed,DataFeed
# import utils
from models import MLP,SimpleLSTM

importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from featfuncs.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from models.ipynb


In [4]:
import anvil.server

In [5]:
LOADDATA=False

In [6]:
OHLC_COLS=['Open_n','High_n','Low_n','Close_n']
OHLC_ORIG=['Open','High','Low','Close']
TA_COLS=['SMA_10', 'SMA_20','VOL_SMA_20','RSI_14','BBL_5_2.0','BBM_5_2.0','BBU_5_2.0',
       'BBB_5_2.0', 'BBP_5_2.0','MACD_12_26_9','MACDh_12_26_9','MACDs_12_26_9']

In [7]:
class MyCustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "__main__":
            module = "mlstrats"
        return super().find_class(module, name)

In [ ]:
class ConfigFields():
    def __init__(self):
        self.flat_features=[]
        self.prev_cols=[]
        self.data_cols=['Open_n','High_n','Low_n','Close_n']+TA_COLS
        self.tar_cols=['top1','top2','top3','bot1','bot2','bot3','(0.02, 0.01)','(0.01, 0.005)','(0.01, 0.02)','(0.005, 0.01)']
#        
#         self.data_cols_to_map=['Open_n','High_n','Low_n','Close_n']
#         self.data_cols_mapped=['Open','High','Low','Close']
#         self.ta_cols=TA_COLS
#         self.data_cols_rest=self.ta_cols
        

In [ ]:
class ModelConfig():
    def __init__(self,config,tickers):
        self.config=config
        self.tickers=tickers

In [ ]:
class MLConfig(ModelConfig):
    def __init__(self,config,mlpshape,lstmshape,MLP6,MLP8,LSTM6,LSTM8,
                 ds_scaler,cs_scaler,xs_scaler,tickers):
        super().__init__(config,tickers)
        self.mlpshape=mlpshape
        self.lstmshape=lstmshape
        self.MLP6dict=MLP6.state_dict()
        self.MLP8dict=MLP8.state_dict()
        self.LSTM6dict=LSTM6.state_dict()
        self.LSTM8dict=LSTM8.state_dict()
        self.ds_scaler=ds_scaler
        self.cs_scaler=cs_scaler
        self.xs_scaler=xs_scaler

In [ ]:
class MLBaseStrat():
    def __init__(self):
        self.logL=[]
        self.train=False
        self.model_type='ml'
    def check_entry(self,df):
        model_results=self.apply_model(df)
        if LOADDATA:
            o1=df.iloc[-1]['(0.02, 0.01)']
            o3=df.iloc[-1]['(0.01, 0.02)']
            print(model_results,o1,o3)
        return self.make_decision(model_results)
    def check_entry_batch(self,dfD):
        decisionsD={}
        log_entry={}
        stopD={t:0 for t in dfD}
        targetD={t:0 for t in dfD}
        for t in dfD.keys():
            model_results=self.apply_model(dfD[t])
            decisionsD[t]=self.make_decision(model_results)
            log_entry[t]=(model_results,dfD[t])
        self.logL+=[log_entry]
        return decisionsD,stopD,targetD
    def check_exit_batch(self,dfD,posf):
        def exit_fn(row):
            return self.exit_predicate(row,dfD[row.ticker])
        posf['to_exit']=posf.apply(exit_fn,axis=1).values
        return posf
    def exit_predicate(self,row,df):
        return False

In [ ]:
class MLStrat(MLBaseStrat):
    def __init__(self,modelConfig=None):
        super().__init__()
        self.init_models(modelConfig=modelConfig)
        self.name='ml-strategy'
    # Model Specific Functions
    def init_models(self,modelConfig=None):
        if modelConfig==None:
            with open('./saved_models/modelConfig.pickle','rb') as f: 
                # self.modelConfig=pickle.load(f)
                unpickler = MyCustomUnpickler(f)
                self.modelConfig=unpickler.load()
        else: self.modelConfig=modelConfig
        self.config=self.modelConfig.config
        self.data_cols=self.config.data_cols
        input_size=len(self.data_cols)
        self.mlpshape=self.modelConfig.mlpshape
        self.MLP6=MLP(dims=self.mlpshape,lr=1e-3,task='classification')
        self.MLP6.load_state_dict(self.modelConfig.MLP6dict)
        self.MLP8=MLP(dims=self.mlpshape,lr=1e-3,task='classification')
        self.MLP8.load_state_dict(self.modelConfig.MLP8dict)
        self.lstmshape=self.modelConfig.lstmshape
        self.LSTM6=SimpleLSTM(input_size=input_size,hidden_sizeL=self.lstmshape,output_size=4,lr=1e-3)
        self.LSTM6.load_state_dict(self.modelConfig.LSTM6dict)
        self.LSTM8=SimpleLSTM(input_size=input_size,hidden_sizeL=self.lstmshape,output_size=4,lr=1e-3)
        self.LSTM8.load_state_dict(self.modelConfig.LSTM8dict)
        self.ds_scaler=self.modelConfig.ds_scaler
        self.cs_scaler=self.modelConfig.cs_scaler
        self.xs_scaler=self.modelConfig.xs_scaler
        self.style='flat'
        self.tickers=self.modelConfig.tickers
        if self.train==False:
            self.MLP6.eval()
            self.MLP8.eval()
            self.LSTM6.eval()
            self.LSTM8.eval()
    def set_style(self,style='flat'):
        self.style=style
    def apply_MLP(self,df):
        s=torch.tensor(df.iloc[-1][self.config.data_cols].values.astype(np.float)).float().unsqueeze(0)
        x=torch.tensor(self.xs_scaler.transform(s)).float()
        r6=self.MLP6(x)
        r8=self.MLP8(x)
        s1=torch.max(r6,1)[1]
        s3=torch.max(r8,1)[1]
        #self.logL+=['MLP',s,s1,s3]
        return s1,s3,r6,r8
    def apply_LSTM(self,df):
        def cs(s):
            return (s.shape[0]*s.shape[1],s.shape[2])
        s=torch.tensor(df[self.config.data_cols].values.astype(np.float)).float().unsqueeze(0)
        x=torch.tensor(self.ds_scaler.transform(s.reshape(cs(s))).reshape(s.shape)).float()
        x=x[:,-40:,:]
        r6=self.LSTM6(x)
        r8=self.LSTM8(x)
        s1=torch.max(r6,1)[1]
        s3=torch.max(r8,1)[1]
        #self.logL+=['LSTM',s,s1,s3]
        return s1,s3,r6,r8,x
    def apply_model(self,df):
        if self.style=='flat':
            s1,s3,r6,r8=self.apply_MLP(df)
        elif self.style=='seq':
            s1,s3,r6,r8,_=self.apply_LSTM(df)
        return s1,s3,r6,r8
    def make_decision(self,model_results):
        s1,s3,_,_=model_results
        if s1==3 or s1==2: return 1
        elif s3==0 or s3==1: return -1
        else: return 0
    def Check(mlstrat,df):
        return mlstrat.check_entry_batch(df)